# Protein feature extraction pipeline

This notebook will contain the pipeline for extracting features from protein sequences. It will be used as a way to show the output without needing to run the `pipeline.py` file locally.

In [23]:
import pyarrow as pa
from fondant.pipeline import Pipeline
from config import MOCK_DATA_PATH_FONDANT

## Loading the dataset

In [24]:
# Create a new pipeline

pipeline = Pipeline(
	name="feature_extraction_pipeline",
	base_path=".fondant",
	description="A pipeline to extract features from protein sequences."
)

## Creating the pipeline

In [25]:
# Read the dataset

dataset = pipeline.read(
	"load_from_parquet",
	arguments={
		"dataset_uri": MOCK_DATA_PATH_FONDANT,
	},
	produces={
		"sequence": pa.string()
	}
)

## Components

This section will contain the components that will be used in the pipeline.

These are the components that will be used in the pipeline:

- `generate_protein_sequence_checksum_component`: This component will generate a checksum for the protein sequence.

- `biopython_component`: This component will extract features from the protein sequence using Biopython.

- `iFeatureOmega_component`: This component will extract features from the protein sequence using iFeature Omega. This component uses arguments to specify the type of features to extract.

- `filter_pdb_component`: This component will filter the PDB files that are already predicted, so the pipeline doesn't need to predict them again. You'll need to specify the following arguments before running the pipeline:
```json
"storage_type": "local",
"pdb_path": "/data/pdb_files/",
"bucket_name": "your-bucket-name",
"project_id": "your-project-id",
"google_cloud_credentials_path": "/data/google_cloud_credentials.json"
```

If you're only using `local`, then you can keep the `bucket_name`, `project_id` and `google_cloud_credentials_path` as empty strings. Using `remote` will require you to have a Google Cloud Storage bucket with credentials and a project ID.

- `predict_protein_3D_structure_component`: This component will predict the 3D structure of the protein using ESMFold.

- `store_pdb_component`: This component will store the PDB files in the provided `storage_type`. You'll need to specify the following arguments before running the pipeline:
```json
"storage_type": "local",
"pdb_path": "/data/pdb_files/",
"bucket_name": "your-bucket-name",
"project_id": "your-project-id",
"google_cloud_credentials_path": "/data/google_cloud_credentials.json"
```

If you're only using `local`, then you can keep the `bucket_name`, `project_id` and `google_cloud_credentials_path` as empty strings. Using `remote` will require you to have a Google Cloud Storage bucket with credentials and a project ID.

In [26]:
import os
import yaml
import json

def create_json_from_yaml(folder_path):
	consumes = {}

	# Walk through all directories and files recursively
	for root, dirs, files in os.walk(folder_path):
		for filename in files:
			if filename == "fondant_component.yaml":
				yaml_file = os.path.join(root, filename)
				# Load YAML data
				with open(yaml_file, 'r') as file:
					yaml_data = yaml.safe_load(file)

				# Extract fields from the produces section
				for key, value in yaml_data.items():
					if key == "produces":
						for obj in value:
							consumes[obj] = obj

	# Convert to JSON
	json_output = json.dumps(consumes)
	return json_output

# Example usage
folder_path = "./components"
all_features = json.loads(create_json_from_yaml(folder_path))

# save to a file
with open('data/all_features.json', 'w') as f:
	json.dump(all_features, f)

In [27]:
all_features

{'sequence': 'sequence',
 'sequence_length': 'sequence_length',
 'molecular_weight': 'molecular_weight',
 'aromaticity': 'aromaticity',
 'isoelectric_point': 'isoelectric_point',
 'instability_index': 'instability_index',
 'gravy': 'gravy',
 'helix': 'helix',
 'turn': 'turn',
 'sheet': 'sheet',
 'charge_at_ph7': 'charge_at_ph7',
 'charge_at_ph5': 'charge_at_ph5',
 'molar_extinction_coefficient_oxidized': 'molar_extinction_coefficient_oxidized',
 'molar_extinction_coefficient_reduced': 'molar_extinction_coefficient_reduced',
 'sequence_checksum': 'sequence_checksum',
 'pdb_string': 'pdb_string',
 'AAC_A': 'AAC_A',
 'AAC_C': 'AAC_C',
 'AAC_D': 'AAC_D',
 'AAC_E': 'AAC_E',
 'AAC_F': 'AAC_F',
 'AAC_G': 'AAC_G',
 'AAC_H': 'AAC_H',
 'AAC_I': 'AAC_I',
 'AAC_K': 'AAC_K',
 'AAC_L': 'AAC_L',
 'AAC_M': 'AAC_M',
 'AAC_N': 'AAC_N',
 'AAC_P': 'AAC_P',
 'AAC_Q': 'AAC_Q',
 'AAC_R': 'AAC_R',
 'AAC_S': 'AAC_S',
 'AAC_T': 'AAC_T',
 'AAC_V': 'AAC_V',
 'AAC_W': 'AAC_W',
 'AAC_Y': 'AAC_Y',
 'APAAC_Pc1.A': 'A

In [33]:
# Apply the components to the dataset

_ = dataset.apply(
	"./components/generate_protein_sequence_checksum_component"
).apply(
	"./components/biopython_component"
).apply(
	"./components/iFeatureOmega_component",
	# currently forcing the number of rows to 5, but there needs to be a better way to do this, see readme for more info
	input_partition_rows=5,
	arguments={
		"descriptors": ["AAC", "GAAC", "Moran", "Geary", "NMBroto", "APAAC"]
	}
).apply(
	"./components/filter_pdb_component",
	arguments={
		"storage_type": "local",
		"pdb_path": "/data/pdb_files",
		"bucket_name": "elated-chassis-400207_dbtl_pipeline_outputs",
		"project_id": "elated-chassis-400207",
		"google_cloud_credentials_path": "/data/google_cloud_credentials.json"
	},
).apply(
	"./components/predict_protein_3D_structure_component",
).apply(
	"./components/store_pdb_component",
	arguments={
		"storage_type": "local",
		"pdb_path": "/data/pdb_files/",
		"bucket_name": "elated-chassis-400207_dbtl_pipeline_outputs",
		"project_id": "elated-chassis-400207",
		"google_cloud_credentials_path": "/data/google_cloud_credentials.json"
	}
)

[2024-03-24 11:30:06,920 | fondant.pipeline.pipeline | WARNING] Component `Generate Protein Sequence Checksum Component` has an image tag set to latest. Caching for the component will be disabled to prevent unpredictable behavior due to images updates
[2024-03-24 11:30:06,927 | fondant.pipeline.pipeline | WARNING] Component `Biopython component` has an image tag set to latest. Caching for the component will be disabled to prevent unpredictable behavior due to images updates
[2024-03-24 11:30:06,963 | fondant.pipeline.pipeline | WARNING] Component `iFeatureOmega component` has an image tag set to latest. Caching for the component will be disabled to prevent unpredictable behavior due to images updates
[2024-03-24 11:30:06,977 | fondant.pipeline.pipeline | WARNING] Component `Filter PDB Component` has an image tag set to latest. Caching for the component will be disabled to prevent unpredictable behavior due to images updates
[2024-03-24 11:30:06,991 | fondant.pipeline.pipeline | WARNING

## Run the pipeline

The `pipeline.py` file needs to be run using the command line. The following command will run the pipeline:

```bash
fondant < full_path_to_pipeline.py >\data:/data
```

But since this is a notebook, the following section will use the `DockerRunner` function of `Fondant` to run the pipeline.

In [34]:
from fondant.pipeline.runner import DockerRunner

DockerRunner().run(input=pipeline, extra_volumes="/data")

[2024-03-24 11:30:09,947 | root | INFO] Found reference to un-compiled pipeline... compiling
[2024-03-24 11:30:09,948 | fondant.pipeline.compiler | INFO] Compiling feature_extraction_pipeline to .fondant/compose.yaml
[2024-03-24 11:30:09,948 | fondant.pipeline.compiler | INFO] Base path found on local system, setting up .fondant as mount volume
[2024-03-24 11:30:09,949 | fondant.pipeline.pipeline | INFO] Sorting pipeline component graph topologically.


InvalidPipelineDefinition: Received a string value for key `sequence` in the `consumes` argument passed to the operation, but `sequence` is not defined in the `consumes` section of the component spec.

## Results

The following results have been taken from the output of the pipeline, which is stored in the `.fondant` directory. This directory contains the output of each component, together with the cache of the previous run.

Currently, the pipeline doesn't implement the `write_to_file` component, so the results will be taken individually from the output of each component.

In [ ]:
import pandas as pd

In [36]:
df_load_from_parquet = pd.read_parquet('./data/export/load_from_parquet/')
df_load_from_parquet.head()

,sequence
id,
0_1,MNQRGMPIQSLVTNVKINRLEENDCIHTRHRVRPGRTDGKNLHAMM...
0_2,MAGLKPEVPLHDGINKFGKSDFAGQEGPKIVTTTDKALLVANGALK...
0_3,MVDLKKELKNFVDSDFPGSPKQEAQGIDVRILLSFNNAAFREALII...
0_4,MELILAKARLEFECDWGLLMLEPCVPPTKIFADRNYAVGVMFESDK...
0_5,MRVLCDGSTGYACAKNTRIRFREKVASVLAKIQGYEQTFPHHMPNM...


In [34]:
df_biopython = pd.read_parquet('./data/export/biopython_component/')
df_biopython.head()

,sequence,sequence_length,molecular_weight,aromaticity,isoelectric_point,instability_index,gravy,helix,turn,sheet,charge_at_ph7,charge_at_ph5,molar_extinction_coefficient_oxidized,molar_extinction_coefficient_reduced
id,,,,,,,,,,,,,,
0_1,MNQRGMPIQSLVTNVKINRLEENDCIHTRHRVRPGRTDGKNLHAMM...,600,66369.0679,0.06,5.397908,38.074,-0.334833,0.385,0.268333,0.328333,-17.260409,9.933956,54320,55070
0_5,MRVLCDGSTGYACAKNTRIRFREKVASVLAKIQGYEQTFPHHMPNM...,420,47355.5634,0.088095,5.392736,42.44,-0.518333,0.319048,0.283333,0.32619,-16.985081,7.337325,58330,58580
0_2,MAGLKPEVPLHDGINKFGKSDFAGQEGPKIVTTTDKALLVANGALK...,400,43254.8112,0.055,5.964593,38.948025,-0.26375,0.335,0.3225,0.335,-6.448485,11.400447,31860,32110
0_3,MVDLKKELKNFVDSDFPGSPKQEAQGIDVRILLSFNNAAFREALII...,550,60158.722,0.072727,5.349652,38.161636,-0.183273,0.341818,0.296364,0.34,-12.666847,6.634677,31860,32110
0_4,MELILAKARLEFECDWGLLMLEPCVPPTKIFADRNYAVGVMFESDK...,350,39615.9422,0.091429,4.825028,40.802857,-0.248,0.38,0.248571,0.357143,-18.025837,-3.682992,61420,61795


In [35]:
df_ifeature_omega_component = pd.read_parquet('./data/export/ifeature_omega_component/')
df_ifeature_omega_component.head()

,sequence,AAC_A,AAC_C,AAC_D,AAC_E,AAC_F,AAC_G,AAC_H,AAC_I,AAC_K,...,NMBroto_BEGF750101.lag3,NMBroto_BEGF750102.lag1,NMBroto_BEGF750102.lag2,NMBroto_BEGF750102.lag3,NMBroto_BEGF750103.lag1,NMBroto_BEGF750103.lag2,NMBroto_BEGF750103.lag3,NMBroto_BHAR880101.lag1,NMBroto_BHAR880101.lag2,NMBroto_BHAR880101.lag3
id,,,,,,,,,,,,,,,,,,,,,
0_1,MNQRGMPIQSLVTNVKINRLEENDCIHTRHRVRPGRTDGKNLHAMM...,0.090000,0.020000,0.048333,0.093333,0.021667,0.081667,0.023333,0.046667,0.056667,...,-0.001961,-0.041237,0.100768,-0.049149,0.086146,0.057148,0.028043,-0.021107,0.125624,-0.004387
0_2,MAGLKPEVPLHDGINKFGKSDFAGQEGPKIVTTTDKALLVANGALK...,0.087500,0.012500,0.050000,0.062500,0.015000,0.080000,0.030000,0.067500,0.060000,...,-0.054463,-0.002630,0.063674,-0.035028,-0.056075,0.069239,-0.028580,0.026617,0.061418,0.047221
0_3,MVDLKKELKNFVDSDFPGSPKQEAQGIDVRILLSFNNAAFREALII...,0.107273,0.009091,0.060000,0.067273,0.043636,0.063636,0.016364,0.067273,0.052727,...,0.087268,0.032190,0.011671,0.006448,0.069078,0.121472,0.136282,0.044959,0.042260,0.005347
0_4,MELILAKARLEFECDWGLLMLEPCVPPTKIFADRNYAVGVMFESDK...,0.071429,0.017143,0.040000,0.108571,0.042857,0.074286,0.014286,0.057143,0.048571,...,-0.016027,0.015219,0.038238,-0.181714,0.066448,0.071851,-0.089194,-0.005339,0.032348,-0.036633
0_5,MRVLCDGSTGYACAKNTRIRFREKVASVLAKIQGYEQTFPHHMPNM...,0.100000,0.011905,0.078571,0.071429,0.033333,0.061905,0.040476,0.061905,0.052381,...,0.038768,0.092617,-0.027349,-0.093597,0.083518,0.039504,-0.037585,0.091911,0.019569,0.001394


> This section uses mock data from already existing pdb files. This is the reason why you'll see 'mock_pdb_string' in the results.

> The pdb files are placed in the `data/pdb_files` directory, which is not included in the repository.

In [31]:
df_filter = pd.read_parquet('./data/export/filter_pdb_component/')
df_filter.head()

,sequence,sequence_checksum,pdb_string
id,,,
0_1,MNQRGMPIQSLVTNVKINRLEENDCIHTRHRVRPGRTDGKNLHAMM...,CRC-94CF2EE011C80480,mock_pdb_string
0_5,MRVLCDGSTGYACAKNTRIRFREKVASVLAKIQGYEQTFPHHMPNM...,CRC-747F108552578E1D,mock_pdb_string
0_2,MAGLKPEVPLHDGINKFGKSDFAGQEGPKIVTTTDKALLVANGALK...,CRC-68D748EC385E9BEC,mock_pdb_string
0_3,MVDLKKELKNFVDSDFPGSPKQEAQGIDVRILLSFNNAAFREALII...,CRC-3B9E0764E7D3C737,mock_pdb_string
0_4,MELILAKARLEFECDWGLLMLEPCVPPTKIFADRNYAVGVMFESDK...,CRC-B08C4E4E86E87F17,mock_pdb_string


In [32]:
df_predict = pd.read_parquet('./data/export/filter_pdb_component/')
df_predict.head()

,sequence,sequence_checksum,pdb_string
id,,,
0_1,MNQRGMPIQSLVTNVKINRLEENDCIHTRHRVRPGRTDGKNLHAMM...,CRC-94CF2EE011C80480,mock_pdb_string
0_5,MRVLCDGSTGYACAKNTRIRFREKVASVLAKIQGYEQTFPHHMPNM...,CRC-747F108552578E1D,mock_pdb_string
0_2,MAGLKPEVPLHDGINKFGKSDFAGQEGPKIVTTTDKALLVANGALK...,CRC-68D748EC385E9BEC,mock_pdb_string
0_3,MVDLKKELKNFVDSDFPGSPKQEAQGIDVRILLSFNNAAFREALII...,CRC-3B9E0764E7D3C737,mock_pdb_string
0_4,MELILAKARLEFECDWGLLMLEPCVPPTKIFADRNYAVGVMFESDK...,CRC-B08C4E4E86E87F17,mock_pdb_string


In [33]:
df_store = pd.read_parquet('./data/export/store_pdb_component/')
df_store.head()

,sequence,sequence_checksum,pdb_string
id,,,
0_1,MNQRGMPIQSLVTNVKINRLEENDCIHTRHRVRPGRTDGKNLHAMM...,CRC-94CF2EE011C80480,mock_pdb_string
0_5,MRVLCDGSTGYACAKNTRIRFREKVASVLAKIQGYEQTFPHHMPNM...,CRC-747F108552578E1D,mock_pdb_string
0_2,MAGLKPEVPLHDGINKFGKSDFAGQEGPKIVTTTDKALLVANGALK...,CRC-68D748EC385E9BEC,mock_pdb_string
0_3,MVDLKKELKNFVDSDFPGSPKQEAQGIDVRILLSFNNAAFREALII...,CRC-3B9E0764E7D3C737,mock_pdb_string
0_4,MELILAKARLEFECDWGLLMLEPCVPPTKIFADRNYAVGVMFESDK...,CRC-B08C4E4E86E87F17,mock_pdb_string
